# Modify the network

deNEST provides a convenient way of modifying the state of some units within a network with the `Layer.set_state` and `Network.set_state` methods

- `Network.set_state` and `Layer.set_state` support __constant__, __multiplicative__ or __additive__ changes (`change_type` parameter)
- We can apply the same change for all units of the layer/population, or provide an array the same shape as the population to perform specific changes for each unit (`from_array` parameter). The array can be directly provided or loaded from file

In this tutorial we:
1. Change the state of units within a single population with the `Layer.set_state` method
    1. Option 1 (`'from_array' == False`): provide a single value , used to change the state of all units of a layer or population
        1. 'constant' changes
        2. 'multiplicative' changes
        3. 'additive' changes
    1. Option 2 (`'from_array' == True`): provide an array of values, mapped to units in the population
        1. User can provide an np-array directly
        2. ..Or  the path to a np-array

1. Change the state of multiple populations at once with the `Network.set_state` method

In [1]:
%load_ext autoreload
%autoreload 2

import nest
import yaml
from pathlib import Path
from pprint import pprint

from denest import *
import denest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
PARAMS_DIR = Path('./data/params/network')

In [3]:
# net.set_state??

## Change the state of units within a population: `Layer.set_state`

In [4]:
nest.ResetKernel()
net = Network(ParamsTree.read(PARAMS_DIR/'network_tree.yml'))
net.create()

2020-06-28 20:26:22,035 [denest.network] INFO: Build N=2 ``Model`` objects
2020-06-28 20:26:22,036 [denest.network] INFO: Build N=2 ``SynapseModel`` objects
2020-06-28 20:26:22,037 [denest.network] INFO: Build N=3 ``Model`` objects
2020-06-28 20:26:22,041 [denest.network] INFO: Build N=2 ``Layer`` or ``InputLayer`` objects.
2020-06-28 20:26:22,049 [denest.utils.validation] INFO: Object `proj_1_AMPA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-28 20:26:22,050 [denest.utils.validation] INFO: Object `proj_2_GABAA`: params: using default value for optional parameters:
{'type': 'topological'}
2020-06-28 20:26:22,053 [denest.network] INFO: Build N=2 ``ProjectionModel`` objects
2020-06-28 20:26:22,057 [denest.network] INFO: Build N=3 ``TopoProjection`` objects
2020-06-28 20:26:22,074 [denest.network] INFO: Build N=2 population recorders.
2020-06-28 20:26:22,081 [denest.network] INFO: Build N=1 projection recorders.
2020-06-28 20:26:22,088 [denest.netw

In [5]:
layer_name = 'l1'
population_name = 'l1_exc'

layer = net.layers['l1']

print('layer shape: ', layer.shape)
print('population shapes: ', layer.population_shape)

layer shape:  (5, 5)
population shapes:  {'l1_exc': (5, 5, 4), 'l1_inh': (5, 5, 2)}


### Option 1: provide a single value, used to change the state of all units of a layer or population

`'from_array'==False` in in `Layer.set_state`

#### "constant" change type

In [6]:
nest_params = {
    'V_m': -69.0,
    'g_peak_AMPA': 0.2,
}

In [7]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-44.0}, 'g_peak_AMPA': {0.1}}
Unique values for l1_inh:  {'V_m': {-55.0}, 'g_peak_AMPA': {0.1}}


In [8]:
# Change param for a single population

layer.set_state(
    nest_params=nest_params,
    population_name='l1_exc',
    change_type='constant',
    from_array=False,
)

2020-06-28 20:26:22,760 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='V_m', from single value')
2020-06-28 20:26:22,761 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='g_peak_AMPA', from single value')


In [9]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-69.0}, 'g_peak_AMPA': {0.2}}
Unique values for l1_inh:  {'V_m': {-55.0}, 'g_peak_AMPA': {0.1}}


In [10]:
# Change param for  all populations

layer.set_state(
    nest_params=nest_params,
    population_name=None,
#     population_name='l1_inh',
    change_type='constant',
    from_array=False,
)

2020-06-28 20:26:23,745 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='V_m', from single value')
2020-06-28 20:26:23,749 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='g_peak_AMPA', from single value')
2020-06-28 20:26:24,563 [denest.network.layers] INFO: Layer='l1', pop='l1_inh': Applying 'constant' change, param='V_m', from single value')
2020-06-28 20:26:24,564 [denest.network.layers] INFO: Layer='l1', pop='l1_inh': Applying 'constant' change, param='g_peak_AMPA', from single value')


In [11]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-69.0}, 'g_peak_AMPA': {0.2}}
Unique values for l1_inh:  {'V_m': {-69.0}, 'g_peak_AMPA': {0.2}}


#### "multiplicative" change 

In [12]:
# Double the value
nest_params = {
    'g_peak_AMPA': 2.0,
}

In [13]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'g_peak_AMPA': {0.2}}
Unique values for l1_inh:  {'g_peak_AMPA': {0.2}}


In [14]:
# Change param for a single population

layer.set_state(
    nest_params=nest_params,
    population_name=None,
    change_type='multiplicative',
    from_array=False,
)

2020-06-28 20:26:25,290 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'multiplicative' change, param='g_peak_AMPA', from single value')
2020-06-28 20:26:26,073 [denest.network.layers] INFO: Layer='l1', pop='l1_inh': Applying 'multiplicative' change, param='g_peak_AMPA', from single value')


In [15]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'g_peak_AMPA': {0.4}}
Unique values for l1_inh:  {'g_peak_AMPA': {0.4}}


#### "additive" change 

In [16]:
# Double the value
nest_params = {
    'V_m': 5.0,
}

In [17]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-69.0}}
Unique values for l1_inh:  {'V_m': {-69.0}}


In [18]:
# Change param for a single population

layer.set_state(
    nest_params=nest_params,
    population_name=None,
    change_type='additive',
    from_array=False,
)

2020-06-28 20:26:26,743 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'additive' change, param='V_m', from single value')
2020-06-28 20:26:27,879 [denest.network.layers] INFO: Layer='l1', pop='l1_inh': Applying 'additive' change, param='V_m', from single value')


In [19]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )
print('Unique values for l1_inh: ', { param: set(nest.GetStatus(layer.gids(population='l1_inh'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-64.0}}
Unique values for l1_inh:  {'V_m': {-64.0}}


### Option 2: provide an array the same shape as the population
... for flexible setting of units state

`'from_array'==True` in in `Layer.set_state`


This can be used to set stimulator state arbitrarily (eg "spike_times" of a spike generator)

#### We can provide the array directly

In [20]:
# Set V_m=-70 for all units except those at location [0, 0]

pop_shape = layer.population_shapes['l1_exc']
V_m_array = -70.0 * np.ones(pop_shape)
V_m_array[0, 0, :] = -60

#### or load the array from file

The 'input_dir' kwarg sets the directory from which arrays are loaded

In [21]:
# Set g_peak_AMPA=0.33 for all units except those at location [0, 0]
g_peak_AMPA_array = 0.33 * np.ones(pop_shape)
g_peak_AMPA_array[0, 0, :] = 1.0

#  save the array to file
INPUT_DIR = Path('./data/input')
os.makedirs(INPUT_DIR, exist_ok=True)
array_path = INPUT_DIR/'g_peak_AMPA_array'
np.save(INPUT_DIR/'g_peak_AMPA_array', g_peak_AMPA_array)

np.load(INPUT_DIR/'g_peak_AMPA_array.npy').shape

(5, 5, 4)

In [22]:
# We provide either the array or the path to an array, relative to the 'input_dir' directory

nest_params = {
    'V_m': V_m_array,
    'g_peak_AMPA': Path('./g_peak_AMPA_array.npy'),
}

In [23]:
print('Unique values for l1_exc: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )

Unique values for l1_exc:  {'V_m': {-64.0}, 'g_peak_AMPA': {0.4}}


In [24]:
# Change param for a single population

layer.set_state(
    nest_params=nest_params,
    input_dir=INPUT_DIR,
    population_name='l1_exc',
    change_type='constant',
    from_array=True,
)

2020-06-28 20:26:28,639 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='V_m', from array')
2020-06-28 20:26:28,643 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='g_peak_AMPA', from array')


In [25]:
print('Unique values for l1_exc at location [0, 0]: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc', location=(0, 0)), param)) for param in nest_params.keys()} )
print('Unique values for l1_exc at all locations: ', { param: set(nest.GetStatus(layer.gids(population='l1_exc'), param)) for param in nest_params.keys()} )

Unique values for l1_exc at location [0, 0]:  {'V_m': {-60.0}, 'g_peak_AMPA': {1.0}}
Unique values for l1_exc at all locations:  {'V_m': {-70.0, -60.0}, 'g_peak_AMPA': {0.33, 1.0}}


## Change the state of units throughout the Network: `Network.set_state`

We can specify modifications for multiple populations at once.

In [26]:
# Set the input layer rates:

input_layer = net.layers['input_layer']
input_layer_shape = input_layer.population_shapes[
    input_layer.stimulator_model
]

# Build the spike times for each unit

spike_times = np.empty(input_layer_shape, dtype=np.object)

# Set the same spike times for all units...
for idx, _ in np.ndenumerate(spike_times):
    spike_times[idx] = [1.0, 10.0]

# Except one unit
idx = (0, 0, 0)
spike_times[idx] = [5.0]

print(spike_times[:,:,0])

[[list([5.0]) list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0])
  list([1.0, 10.0])]
 [list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0])
  list([1.0, 10.0])]
 [list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0])
  list([1.0, 10.0])]
 [list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0])
  list([1.0, 10.0])]
 [list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0]) list([1.0, 10.0])
  list([1.0, 10.0])]]


In [27]:
net.set_state(
    [
        {
            'layers': ['l1'],
            'population_name': None,
            'change_type': 'constant',
            'from_array': False,
            'nest_params': {
                'V_m': -69.9
            }
            
        },
        {
            'layers': ['input_layer'],
            'population_name': 'spike_generator',
            'change_type': 'constant',
            'from_array': True,
            'nest_params': {
                'spike_times': spike_times,
            }
            
        },
        
    ]
)

2020-06-28 20:26:30,070 [denest.network.layers] INFO: Layer='input_layer', pop='spike_generator': Applying 'constant' change, param='spike_times', from array')
2020-06-28 20:26:30,138 [denest.network.layers] INFO: Layer='l1', pop='l1_exc': Applying 'constant' change, param='V_m', from single value')
2020-06-28 20:26:31,030 [denest.network.layers] INFO: Layer='l1', pop='l1_inh': Applying 'constant' change, param='V_m', from single value')


In [28]:
# Get status of spike generators
print(
    nest.GetStatus(net.layers['input_layer'].gids(population='spike_generator'), 'spike_times')
)

(array([5.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]), array([ 1., 10.]))


In [29]:
# Get status of l1 units
print(
    nest.GetStatus(net.layers['l1'].gids(), 'V_m')
)

(-69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9, -69.9